In [1]:
import os

In [2]:
%pwd

'/Users/tejasredkar/Developer/Chest-Cancer-Prediction-using-ML-Flow-DVC/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/tejasredkar/Developer/Chest-Cancer-Prediction-using-ML-Flow-DVC'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/tejred213/Chest-Cancer-Prediction-using-ML-Flow-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="tejred213"
os.environ["MLFLOW_TRACKING_PASSWORD"]="bf81a71e3c7bbf104640edcc945d285b17ae08e3"

In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/tejred213/Chest-Cancer-Prediction-using-ML-Flow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [11]:
# %pip install mlflow

import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1.0 / 255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        # Fixed `model` to `self.model`
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Handle model logging based on registry type
            try:
                if tracking_url_type_store != "file":
                    mlflow.keras.log_model(
                        self.model, "model", registered_model_name="VGG16Model"
                    )
                else:
                    mlflow.keras.log_model(self.model, "model")
            except ModuleNotFoundError as e:
                # Replace distutils usage with setuptools if possible
                if "distutils" in str(e):
                    print("Switching to setuptools due to distutils issue.")
                    import setuptools  # Ensure setuptools is installed
                else:
                    raise e


# Main execution
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except ModuleNotFoundError as e:
    if "distutils" in str(e):
        print("Distutils is missing. Please install setuptools and try again.")
        import subprocess
        subprocess.check_call(["pip", "install", "setuptools"])
    else:
        raise e

except Exception as e:
    raise e


Found 306 images belonging to 2 classes.
20/20 [==============================] - 7s 332ms/step - loss: 0.5892 - accuracy: 0.7582


2025/01/27 19:59:14 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/br/8xndxmrd1p74dp71tt814grw0000gn/T/tmpoqbgdjeq/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/br/8xndxmrd1p74dp71tt814grw0000gn/T/tmpoqbgdjeq/model/data/model/assets
2025/01/27 19:59:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/01/27 19:59:47 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 6
Created version '6' of model 'VGG16Model'.


🏃 View run honorable-bear-696 at: https://dagshub.com/tejred213/Chest-Cancer-Prediction-using-ML-Flow-DVC.mlflow/#/experiments/0/runs/67202e9c58b3460686a07c188f2237f7
🧪 View experiment at: https://dagshub.com/tejred213/Chest-Cancer-Prediction-using-ML-Flow-DVC.mlflow/#/experiments/0


In [15]:
# class Evaluation:
#     def __init__(self, config: EvaluationConfig):
#         self.config = config

#     def _valid_generator(self):
#         datagenerator_kwargs = dict(
#             rescale=1.0 / 255,
#             validation_split=0.30
#         )

#         dataflow_kwargs = dict(
#             target_size=self.config.params_image_size[:-1],
#             batch_size=self.config.params_batch_size,
#             interpolation="bilinear"
#         )

#         valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
#             **datagenerator_kwargs
#         )

#         self.valid_generator = valid_datagenerator.flow_from_directory(
#             directory=self.config.training_data,
#             subset="validation",
#             shuffle=False,
#             **dataflow_kwargs
#         )

#     @staticmethod
#     def load_model(path: Path) -> tf.keras.Model:
#         return tf.keras.models.load_model(path)

#     def evaluation(self):
#         self.model = self.load_model(self.config.path_of_model)
#         self._valid_generator()
#         # Fixed `model` to `self.model`
#         self.score = self.model.evaluate(self.valid_generator)
#         self.save_score()

#     def save_score(self):
#         scores = {"loss": self.score[0], "accuracy": self.score[1]}
#         save_json(path=Path("scores.json"), data=scores)

#     def log_into_mlflow(self):
#         mlflow.set_registry_uri(self.config.mlflow_uri)
#         tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

#         with mlflow.start_run():
#             mlflow.log_params(self.config.all_params)
#             mlflow.log_metrics(
#                 {"loss": self.score[0], "accuracy": self.score[1]}
#             )
#             # Model registry does not work with file store
#             if tracking_url_type_store != "file":
#                 # Register the model
#                 mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
#             else:
#                 mlflow.keras.log_model(self.model, "model")


In [ ]:
# try:
#     config = ConfigurationManager()
#     eval_config = config.get_evaluation_config()
#     evaluation = Evaluation(eval_config)
#     evaluation.evaluation()
#     evaluation.log_into_mlflow()

# except Exception as e:
#    raise e